### Import required libraries

In [ ]:
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier

### Define a custom confusion matrix illustrator

In [ ]:
def show_confusion_matrix(y_test, y_pred):
    labels = ["tDCS", "sham", "control"]
    cf_matrix = confusion_matrix(y_test, y_pred)
    cmap = sns.diverging_palette(220, 10, as_cmap=True)
    sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, fmt='.2%', cmap=cmap, cbar=False, yticklabels=labels, xticklabels=labels)
    plt.ylabel('True label')
    plt.xlabel('Predicted Label')

### Load the data frame from the pickle file

In [ ]:
with open("data\DataFrame.pickle", "rb") as file:
    x, y = pickle.load(file, )

### Split the data frame into two sets: train and test

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

### Define some variable for collecting results of the models

In [ ]:
responses = {}

### *k-nearest neighbors(KNN) Model*

In [ ]:
accuracies_score = []
for n_index in range(1, len(y_train)-1):
    KNNModel = KNeighborsClassifier(n_neighbors=n_index)
    KNNModel.fit(x_train, y_train)
    y_pred = KNNModel.predict(x_test)
    accuracies_score.append(accuracy_score(y_test, y_pred))

n_index=accuracies_score.index(max(accuracies_score))+1
KNNModel = KNeighborsClassifier(n_neighbors=n_index)
KNNModel.fit(x_train, y_train)
y_pred = KNNModel.predict(x_test)
responses["KNN"] = y_pred

#### *Random Forest Classifier Model*

In [ ]:
RFCModel = RandomForestClassifier(n_jobs=-1)
RFCModel.fit(x_train, y_train)
y_pred = RFCModel.predict(x_test)
responses["Random Forest"] = y_pred

### *Logistic Regression Model*

In [ ]:
LRModel = LogisticRegression(solver="liblinear", max_iter=30000)
LRModel.fit(x_train, y_train)
y_pred = LRModel.predict(x_test)
responses["Logistic Regression"] = y_pred

### *Support Vector Machine Classifier Model*

In [ ]:
SVCModel = SVC()
SVCModel.fit(x_train, y_train)
y_pred = SVCModel.predict(x_test)
responses["SVC"] = y_pred

### *XGBoost Model*

In [ ]:
XGBModel = XGBClassifier()
XGBModel.fit(x_train, y_train)
y_pred = XGBModel.predict(x_test)
responses["XGBoost"] = y_pred

### Ridge

In [ ]:
RidgeModel = RidgeClassifier()
RidgeModel.fit(x_train, y_train)
y_pred = RidgeModel.predict(x_test)
responses["Ridge"] = y_pred

### *Catboost*

In [ ]:
CBCModel = CatBoostClassifier()
CBCModel.fit(x_train, y_train)
y_pred = CBCModel.predict(x_test)
responses["CatBoost"] = y_pred

### *Multi-layer Perceptron classifier(MLP classifier)*

In [ ]:
MLPCModel = MLPClassifier()
MLPCModel.fit(x_train, y_train)
y_pred = MLPCModel.predict(x_test)
responses["MLP"] = y_pred

### Show confusion matrix of each model

In [ ]:
plot_index=1
for response in responses:
    plt.figure("{0}".format(plot_index), figsize=[8, 8])
    plt.title(response)
    show_confusion_matrix(y_test, responses[response])
    plot_index+=1

In [ ]:
for response in responses:
    print("{0} :".format(response))
    print(classification_report(y_test, responses[response], zero_division=0))